In [1]:
from zeugma.embeddings import EmbeddingTransformer

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

In [3]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from preprocessing import *
from function import *

In [5]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from deslib.util.aggregation import *

In [6]:
df_train = pd.read_csv("./Data/Hateval/hateval2019_en_train.csv").dropna()
df_val = pd.read_csv("./Data/Hateval/hateval2019_en_dev.csv").dropna()
df_test = pd.read_csv("./Data/Hateval/hateval2019_en_test.csv").dropna()

In [7]:
df_test.head()

,id,text,HS,TR,AG
0,34243,"@local1025 @njdotcom @GovMurphy Oh, I could ha...",0,0,0
1,30593,Several of the wild fires in #california and #...,0,0,0
2,31427,@JudicialWatch My question is how do you reset...,0,0,0
3,31694,"#Europe, you've got a problem! We must hurry...",1,0,0
4,31865,This is outrageous! #StopIllegalImmigration #...,1,0,0


In [8]:
train, class_train = df_train['text'].apply(pre_processing), df_train['HS']
val, class_val = df_val['text'].apply(pre_processing), df_val['HS']
test, class_test = df_test['text'].apply(pre_processing), df_test['HS']

In [9]:
w2v = EmbeddingTransformer('word2vec')

In [10]:
glove = EmbeddingTransformer('glove')

In [11]:
fasttext = EmbeddingTransformer('fasttext')

In [12]:
cv = CountVectorizer(analyzer='word', lowercase=True, stop_words='english')
cv.fit_transform(train.values.astype('U'))
     
tfidf =  TfidfVectorizer(analyzer='word', lowercase=True, use_idf=True, stop_words='english')
tfidf.fit_transform(train.values.astype('U'))

<9000x16056 sparse matrix of type '<class 'numpy.float64'>'
	with 93433 stored elements in Compressed Sparse Row format>

## Prediction SVM base model

In [13]:
svm = {
    'CV': {
        'CLF': SVC(random_state=42, kernel='linear', gamma=0.1, probability=True),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': SVC(random_state=42, kernel='linear', gamma=0.1, probability=True),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': SVC(random_state=42, kernel='rbf', gamma=1, probability=True),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': SVC(random_state=42, kernel='rbf', gamma=0.5, probability=True),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': SVC(random_state=42, kernel='rbf', gamma=1, probability=True),
        'EXT': fasttext,
    }
}

In [14]:
svm_clfs = {

}
for ext, clf in svm.items():
    svm_clfs[ext] = get_classifier(clf['CLF'], train, class_train, clf['EXT'])

In [15]:
df_pred_train = pd.DataFrame(df_train['HS'])
df_pred_val = pd.DataFrame(df_val['HS'])
df_pred_test = pd.DataFrame(df_test['HS'])

df_prob_val = pd.DataFrame(df_val['HS'])
df_prob_train = pd.DataFrame(df_train['HS'])
df_prob_test = pd.DataFrame(df_test['HS'])

for ext, clf in svm_clfs.items():
  # Predict
    df_train_ = pd.DataFrame(clf.predict(train), columns=["SVM-{}".format(ext)])
    df_val_ = pd.DataFrame(clf.predict(val), columns=["SVM-{}".format(ext)])
    df_test_ = pd.DataFrame(clf.predict(test), columns=["SVM-{}".format(ext)])
  
    df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
    df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
    df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)
  
  # # Probabilidades
    cols = [
      "SVM-{}-{}".format(ext,clf.classes_[0]), 
      "SVM-{}-{}".format(ext,clf.classes_[1])
    ]
    df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
    df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
    df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
    df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
    df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
    df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


## Prediction LR model

In [16]:
lr = {
    'CV': {
        'CLF': LogisticRegression(random_state=42, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': LogisticRegression(random_state=42, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': LogisticRegression(random_state=42, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': LogisticRegression(random_state=42, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': LogisticRegression(random_state=42, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT': fasttext,
    }
}

In [17]:
lr_clfs = {

}
for ext, clf in lr.items():
    lr_clfs[ext] = get_classifier(clf['CLF'], train, df_train['HS'], clf['EXT'])

In [18]:
for ext, clf in lr_clfs.items():
  # Predict
    df_train_ = pd.DataFrame(clf.predict(train), columns=["LR-{}".format(ext)])
    df_val_ = pd.DataFrame(clf.predict(val), columns=["LR-{}".format(ext)])
    df_test_ = pd.DataFrame(clf.predict(test), columns=["LR-{}".format(ext)])
  
    df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
    df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
    df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

    # # Probabilidades
    cols = [
      "LR-{}-{}".format(ext,clf.classes_[0]), 
      "LR-{}-{}".format(ext,clf.classes_[1])
    ]
    df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
    df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
    df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
    df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
    df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
    df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

## Prediction Random Forest

In [19]:
rf = {
    'CV': {
        'CLF': RandomForestClassifier(random_state=42, n_estimators=100, n_jobs=-1),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': RandomForestClassifier(random_state=42, n_estimators=100, n_jobs=-1),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': RandomForestClassifier(random_state=42, n_estimators=100, n_jobs=-1),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': RandomForestClassifier(random_state=42, n_estimators=100, n_jobs=-1),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': RandomForestClassifier(random_state=42, n_estimators=100, n_jobs=-1),
        'EXT': fasttext,
    }
}

In [20]:
rf_clfs = {

}
for ext, clf in rf.items():
    rf_clfs[ext] = get_classifier(clf['CLF'], train, df_train['HS'], clf['EXT'])

In [21]:
for ext, clf in rf_clfs.items():
    # Predict
    df_train_ = pd.DataFrame(clf.predict(train), columns=["RF-{}".format(ext)])
    df_val_ = pd.DataFrame(clf.predict(val), columns=["RF-{}".format(ext)])
    df_test_ = pd.DataFrame(clf.predict(test), columns=["RF-{}".format(ext)])
  
    df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
    df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
    df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

    # # Probabilidades
    cols = [
      "RF-{}-{}".format(ext,clf.classes_[0]), 
      "RF-{}-{}".format(ext,clf.classes_[1])
    ]
    df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
    df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
    df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
    df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
    df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
    df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


### PRediction Naive Bayes

In [22]:
nb = {
    'CV': {
        'CLF': MultinomialNB(alpha=1, fit_prior=False),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': MultinomialNB(alpha=0.5, fit_prior=False),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': GaussianNB(priors=[0.5, 0.5]),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': GaussianNB(priors=[0.5, 0.5]),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': GaussianNB(priors=[0.5, 0.5]),
        'EXT': fasttext,
    }
}

In [23]:
nb_clfs = {

}
for ext, clf in nb.items():
    nb_clfs[ext] = get_classifier(clf['CLF'], train, df_train['HS'], clf['EXT'])

In [24]:
for ext, clf in nb_clfs.items():
    # Predict
    df_train_ = pd.DataFrame(clf.predict(train), columns=["NB-{}".format(ext)])
    df_val_ = pd.DataFrame(clf.predict(val), columns=["NB-{}".format(ext)])
    df_test_ = pd.DataFrame(clf.predict(test), columns=["NB-{}".format(ext)])
  
    df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
    df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
    df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
    cols = [
      "NB-{}-{}".format(ext,clf.classes_[0]), 
      "NB-{}-{}".format(ext,clf.classes_[1])
    ]
    df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
    df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
    df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
    df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
    df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
    df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

### PRediction MLP

In [25]:
mlp = {
    'CV': {
        'CLF': MLPClassifier(random_state=42, batch_size=64, max_iter=100, activation='relu', solver='lbfgs'),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': MLPClassifier(random_state=42, batch_size=64, max_iter=100, activation='logistic', solver='adam'),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': MLPClassifier(random_state=42, batch_size=64, max_iter=100, activation='relu', solver='adam'),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': MLPClassifier(random_state=42, batch_size=64, max_iter=100, activation='relu', solver='adam'),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=20, activation='relu', solver='adam'),
        'EXT': fasttext,
    }
}

In [26]:
mlp_clfs = {

}
for ext, clf in mlp.items():
    mlp_clfs[ext] = get_classifier(clf['CLF'], train, df_train['HS'], clf['EXT'])

C:\Users\Rafael\Anaconda3\envs\hate_speech\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Rafael\Anaconda3\envs\hate_speech\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Rafael\Anaconda3\envs\hate_speech\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Rafael\Anaconda3\envs\hate_speech\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) rea

In [27]:
for ext, clf in mlp_clfs.items():
    # Predict
    df_train_ = pd.DataFrame(clf.predict(train), columns=["MLP-{}".format(ext)])
    df_val_ = pd.DataFrame(clf.predict(val), columns=["MLP-{}".format(ext)])
    df_test_ = pd.DataFrame(clf.predict(test), columns=["MLP-{}".format(ext)])

    df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
    df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
    df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

    # # Probabilidades
    cols = [
        "MLP-{}-{}".format(ext,clf.classes_[0]), 
        "MLP-{}-{}".format(ext,clf.classes_[1])
    ]
    df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
    df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
    df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
    df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
    df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
    df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


## Prediction Extra

In [28]:
extra = {
    'CV': {
        'CLF': ExtraTreesClassifier(random_state=42, n_estimators=100, n_jobs=-1),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': ExtraTreesClassifier(random_state=42, n_estimators=100, n_jobs=-1),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': ExtraTreesClassifier(random_state=42, n_estimators=100, n_jobs=-1),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': ExtraTreesClassifier(random_state=42, n_estimators=100, n_jobs=-1),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': ExtraTreesClassifier(random_state=42, n_estimators=100, n_jobs=-1),
        'EXT': fasttext,
    }
}

In [29]:
extra_clfs = {

}
for ext, clf in extra.items():
    extra_clfs[ext] = get_classifier(clf['CLF'], train, df_train['HS'], clf['EXT'])


In [30]:
for ext, clf in extra_clfs.items():
    # Predict
    df_train_ = pd.DataFrame(clf.predict(train), columns=["EXTRA-{}".format(ext)])
    df_val_ = pd.DataFrame(clf.predict(val), columns=["EXTRA-{}".format(ext)])
    df_test_ = pd.DataFrame(clf.predict(test), columns=["EXTRA-{}".format(ext)])

    df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
    df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
    df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

    # # Probabilidades
    cols = [
        "EXTRA-{}-{}".format(ext,clf.classes_[0]), 
        "EXTRA-{}-{}".format(ext,clf.classes_[1])
    ]
    df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
    df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
    df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
    df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
    df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
    df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [31]:
knn = {
    'CV': {
        'CLF': KNeighborsClassifier(n_neighbors=3, algorithm='auto'),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': KNeighborsClassifier(n_neighbors=5, algorithm='auto'),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': KNeighborsClassifier(n_neighbors=5, algorithm='auto'),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': KNeighborsClassifier(n_neighbors=5, algorithm='auto'),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': KNeighborsClassifier(n_neighbors=5, algorithm='auto'),
        'EXT': fasttext,
    }
}

In [32]:
knn_clfs = { }
for ext, clf in knn.items():
    knn_clfs[ext] = get_classifier(clf['CLF'], train, df_train['HS'], clf['EXT'])

In [33]:
for ext, clf in knn_clfs.items():
    # Predict
    df_train_ = pd.DataFrame(clf.predict(train), columns=["KNN-{}".format(ext)])
    df_val_ = pd.DataFrame(clf.predict(val), columns=["KNN-{}".format(ext)])
    df_test_ = pd.DataFrame(clf.predict(test), columns=["KNN-{}".format(ext)])

    df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
    df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
    df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

    # # Probabilidades
    cols = [
        "KNN-{}-{}".format(ext,clf.classes_[0]), 
        "KNN-{}-{}".format(ext,clf.classes_[1])
    ]
    df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
    df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
    df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
    df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
    df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
    df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


# Prediction CNN

In [34]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH=300

y_train = to_categorical(df_train['HS'])
y_val = to_categorical(df_val['HS'])

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(train)

seq_train = tokenizer.texts_to_sequences(train)
seq_val = tokenizer.texts_to_sequences(val)
seq_test = tokenizer.texts_to_sequences(test)

data_train = pad_sequences(seq_train, maxlen=MAX_SEQUENCE_LENGTH)
data_val = pad_sequences(seq_val, maxlen=MAX_SEQUENCE_LENGTH)
data_test = pad_sequences(seq_test, maxlen=MAX_SEQUENCE_LENGTH)

In [35]:
cnn_cv = get_CNN(cv, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', dense=2)
cnn_tfidf = get_CNN(tfidf, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='softmax', dense=2)
cnn_w2v = get_CNN(w2v, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True, dense=2)
cnn_glove = get_CNN(glove, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=25, activation='sigmoid', word_embedding=True, dense=2)
cnn_fast = get_CNN(fasttext, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True, dense=2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 300)          5142600   
_________________________________________________________________
dropout (Dropout)            (None, 300, 300)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 298, 64)           57664     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 256)               16640     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
activation (Activation)      (None, 256)               0

In [36]:
cnn_cv.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=20)

Epoch 1/20
450/450 [==============================] - 18s 39ms/step - loss: 0.6796 - accuracy: 0.5792 - val_loss: 0.6560 - val_accuracy: 0.6570
Epoch 2/20
450/450 [==============================] - 17s 38ms/step - loss: 0.6228 - accuracy: 0.6704 - val_loss: 0.6083 - val_accuracy: 0.6910
Epoch 3/20
450/450 [==============================] - 18s 40ms/step - loss: 0.5637 - accuracy: 0.7221 - val_loss: 0.5975 - val_accuracy: 0.7000
Epoch 4/20
450/450 [==============================] - 17s 38ms/step - loss: 0.5368 - accuracy: 0.7431 - val_loss: 0.6592 - val_accuracy: 0.6780
Epoch 5/20
450/450 [==============================] - 17s 38ms/step - loss: 0.5130 - accuracy: 0.7497 - val_loss: 0.6216 - val_accuracy: 0.7010
Epoch 6/20
450/450 [==============================] - 17s 38ms/step - loss: 0.4835 - accuracy: 0.7739 - val_loss: 0.5607 - val_accuracy: 0.7210
Epoch 7/20
450/450 [==============================] - 17s 38ms/step - loss: 0.4685 - accuracy: 0.7828 - val_loss: 0.5613 - val_accuracy:

In [37]:
cnn_tfidf.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)
cnn_w2v.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)
cnn_glove.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)
cnn_fast.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)

Epoch 1/20
45/45 [==============================] - 15s 316ms/step - loss: 0.6841 - accuracy: 0.5668 - val_loss: 0.6815 - val_accuracy: 0.5730
Epoch 2/20
45/45 [==============================] - 14s 313ms/step - loss: 0.6779 - accuracy: 0.5774 - val_loss: 0.6720 - val_accuracy: 0.5730
Epoch 3/20
45/45 [==============================] - 14s 313ms/step - loss: 0.6601 - accuracy: 0.5973 - val_loss: 0.6520 - val_accuracy: 0.6140
Epoch 4/20
45/45 [==============================] - 14s 314ms/step - loss: 0.6340 - accuracy: 0.6414 - val_loss: 0.6295 - val_accuracy: 0.6430
Epoch 5/20
45/45 [==============================] - 14s 320ms/step - loss: 0.6033 - accuracy: 0.6944 - val_loss: 0.6134 - val_accuracy: 0.6810
Epoch 6/20
45/45 [==============================] - 14s 315ms/step - loss: 0.5776 - accuracy: 0.7172 - val_loss: 0.6068 - val_accuracy: 0.6810
Epoch 7/20
45/45 [==============================] - 14s 312ms/step - loss: 0.5632 - accuracy: 0.7263 - val_loss: 0.6081 - val_accuracy: 0.6680

45/45 [==============================] - 3s 68ms/step - loss: 0.5351 - accuracy: 0.7336 - val_loss: 0.5823 - val_accuracy: 0.6910
Epoch 19/20
45/45 [==============================] - 3s 68ms/step - loss: 0.5189 - accuracy: 0.7373 - val_loss: 0.5818 - val_accuracy: 0.6960
Epoch 20/20
45/45 [==============================] - 3s 70ms/step - loss: 0.5304 - accuracy: 0.7276 - val_loss: 0.5850 - val_accuracy: 0.6950
Epoch 1/20
45/45 [==============================] - 18s 378ms/step - loss: 0.6710 - accuracy: 0.5941 - val_loss: 0.6251 - val_accuracy: 0.6400
Epoch 2/20
45/45 [==============================] - 17s 385ms/step - loss: 0.5979 - accuracy: 0.6845 - val_loss: 0.5949 - val_accuracy: 0.6740
Epoch 3/20
45/45 [==============================] - 14s 313ms/step - loss: 0.5468 - accuracy: 0.7226 - val_loss: 0.5735 - val_accuracy: 0.6910
Epoch 4/20
45/45 [==============================] - 14s 314ms/step - loss: 0.5153 - accuracy: 0.7422 - val_loss: 0.5649 - val_accuracy: 0.7010
Epoch 5/20
45/

In [38]:
# df_pred_test = df_pred_test.drop(['CNN-CV'], axis=1)
# print(df_pred_test.columns)
# CV
cols = ["CNN-CV-0", "CNN-CV-1"]
df_train_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_train), axis=1), columns=["CNN-CV"])
df_val_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_val), axis=1), columns=["CNN-CV"])
df_test_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_test), axis=1), columns=["CNN-CV"])
  
df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# print(df_pred_test.columns)

# # Probabilidades
  
df_train_ = pd.DataFrame(cnn_cv.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_cv.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_cv.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


cols = ["CNN-TF-0", "CNN-TF-1"]

df_train_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_train), axis=1), columns=["CNN-TFIDF"])
df_val_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_val), axis=1), columns=["CNN-TFIDF"])
df_test_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_test), axis=1), columns=["CNN-TFIDF"])
  
df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# Probabilidades
  
df_train_ = pd.DataFrame(cnn_tfidf.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_tfidf.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_tfidf.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

cols = ["CNN-W2V-0", "CNN-W2V-1"]

df_train_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_train), axis=1), columns=["CNN-W2V"])
df_val_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_val), axis=1), columns=["CNN-W2V"])
df_test_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_test), axis=1), columns=["CNN-W2V"])
  
df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# Probabilidades
  
df_train_ = pd.DataFrame(cnn_w2v.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_w2v.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_w2v.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


cols = ["CNN-GLOVE-0", "CNN-GLOVE-1"]

df_train_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_train), axis=1), columns=["CNN-GLOVE"])
df_val_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_val), axis=1), columns=["CNN-GLOVE"])
df_test_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_test), axis=1), columns=["CNN-GLOVE"])

df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# Probabilidades
  
df_train_ = pd.DataFrame(cnn_glove.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_glove.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_glove.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

cols = ["CNN-FAST-0", "CNN-FAST-1"]

df_train_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_train), axis=1), columns=["CNN-FAST"])
df_val_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_val), axis=1), columns=["CNN-FAST"])
df_test_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_test), axis=1), columns=["CNN-FAST"])

df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# Probabilidades
  
df_train_ = pd.DataFrame(cnn_fast.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_fast.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_fast.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [39]:
df_pred_train.to_csv("pred_train_hateval.csv")
df_pred_val.to_csv("pred_val_hateval.csv")
df_pred_test.to_csv("pred_test_hateval.csv")

In [40]:
df_prob_train.to_csv("prob_train_hateval.csv")
df_prob_val.to_csv("prob_val_hateval.csv")
df_prob_test.to_csv("prob_test_hateval.csv")